# Initializing setup

In [1]:
import os
import time
import uuid
import cv2

# Collecting images

In [4]:
TRAIN_PATH = 'real_data','train','images'
TEST_PATH = 'real_data','test','images'
VAL_PATH = 'real_data','val','images'